In [146]:
import urllib.parse

from enum import Enum

class Environment(Enum):
    PRODUCTION = "serlo.org"
    STAGING = "serlo-staging.dev"
    
class SerloEnvironment:
    def __init__(self, env=Environment.STAGING):
        self.env = env
        
    @property
    def domain(self):
        return self.env.value
    
    def get_url(self, scheme="https", subdomain="", path="/", query=""):
        return urllib.parse.urlunparse((scheme, self.get_netloc(subdomain), path, "", query, ""))
    
    def get_auth(self):
        return ("serloteam", "serloteam") if self.env == Environment.STAGING else None
    
    def get_netloc(self, subdomain):
        return ".".join([subdomain, self.domain]) if subdomain else self.domain

print(SerloEnvironment().get_url(subdomain="api"))
print(SerloEnvironment().get_url(path="/a"))
print(SerloEnvironment().get_auth())

https://api.serlo-staging.dev/
https://serlo-staging.dev/a
('serloteam', 'serloteam')


In [172]:
import requests
import urllib.parse

def get_domain(url):
    netloc = urllib.parse.urlparse(url).netloc
    
    return ".".join(netloc.split(".")[-2:])

def get_subdomain(url):
    netloc = urllib.parse.urlparse(url).netloc
    
    return ".".join(netloc.split(".")[:-2])
        
print(get_domain("http://api.serlo-staging.org/p"))
print(get_subdomain("http://api.serlo-staging.org/p"))
print(len(get_subdomain("http://serlo-staging.org/p")))

serlo-staging.org
api
0


In [177]:
from getpass import getpass

username = getpass("Username")
password = getpass("Password")

Username········
Password········


In [179]:
import requests

import json
import logging
from bs4 import BeautifulSoup

from http.client import HTTPConnection

# print statements from `http.client.HTTPConnection` to console/stdout
HTTPConnection.debuglevel = 0

class SerloSession(requests.Session):
    def __init__(self, env):
        self.env = env
        
        super().__init__()
        
    def prepare_request(self, request):
        if self.is_serlo_domain(request.url) and get_subdomain(request.url) in ["en", "de", "es", "hi", "ta"]:
            request.auth = self.env.get_auth()
        
        return super().prepare_request(request)
    
    def should_strip_auth(self, old_url, new_url):
        if self.is_serlo_domain(old_url) and self.is_serlo_domain(new_url):
            return False
        
        return super().should_strip_auth(old_url, new_url)
    
    def is_serlo_domain(self, url):
        return get_domain(url) == self.env.domain

class SerloBot:
    def __init__(self, env=Environment.STAGING):
        self.env = SerloEnvironment(env)
        self.session = SerloSession(self.env)
    
    def login(self, username, password):
        login_page = self.session.get(self.env.get_url(subdomain="en", path="/api/auth/login"),
                                      headers = {"Referer": self.env.get_url(subdomain="en")})
        
        login_page.raise_for_status()
        login_form = BeautifulSoup(login_page.text).find("form", id="login")
        
        after_login = self.session.post(login_page.url, data={
            "csrf": login_form.select('input[name="csrf"]')[0]["value"],
            "login_challenge": login_form.select('input[name="login_challenge"]')[0]["value"],
            "email": username,
            "password": password,
            "submit": "Login",
            "remember": ["0", "1"]
        })
        
        after_login.raise_for_status()
    
    def api_call(self, query, variables={}):
        headers = {
            "Content-Type": "application/json",
            "Referer": self.env.get_url(subdomain="en")
        }
        access_token = self.get_access_token()
        
        if access_token:
            headers["Authorization"] = "Bearer " + access_token
        
        response = self.session.post(self.env.get_url(subdomain="api", path="/graphql"),
            headers = headers,
            json = { "query": query, "variables": variables })
        
        if response.status_code not in [200,400]:
            response.raise_for_status()
        
        result = response.json()
        
        if "errors" in result and result["errors"]:
            raise requests.HTTPError(result["errors"][0]["message"], response=response)
        
        return result["data"]
    
    def get_access_token(self):
        auth_token = self.session.cookies.get("auth-token", domain=self.env.get_netloc("en"))
        
        if auth_token != None:
            try:
                return json.loads(auth_token).get("access_token", None)
            except json.JSONDecodeError:
                return None
        
        return None
    
bot = SerloBot(Environment.PRODUCTION)
bot.login(username, password)
bot.api_call(query="""
        mutation {
            user {
                deleteBots(input: {botIds: [234104]}) {
                    success
                }
            }
        }
    """)

{'user': {'deleteBots': {'success': True}}}